# Отображение метрик в визуальном виде и генерация данных 
Программа вызывает мое приложение, написанное на с++, то в свою очередь принимает массив на вход и возращает отсорированный массив и время выполенения  

In [ ]:
288832050635919849428592395300697146420447483647024794528
2888320509218434389168198289024794528

In [ ]:
5950214967818727437310749423066932998 - правильный
595072135467891234567891234567891234567  - квен 
5950214967818727437310749423066932998 - гугл 